# Calibration

This section of the tutorial will cover the calibration module of Mitiq. 

Before running this tutorial, make sure you have `mitiq` installed.
```
pip install mitiq
```
Use `pip list | grep mitiq` to ensure it is installed.

In [8]:
# If you have not already done so, you can instll mitiq by uncommenting out the line below
# %pip install mitiq

In [31]:
from mitiq import (
    Calibrator,
    SUPPORTED_PROGRAM_TYPES,
    MeasurementResult
)
from mitiq.calibration import PEC_SETTINGS

import numpy as np
import cirq
from qiskit_ibm_runtime.fake_provider import FakeJakartaV2  # Fake (simulated) QPU

## Executor for Calibration
When using the calibration module to test different mitigation techniques, you must define an executor that returns measured bitstrings. 

In [23]:
def execute_cirq_circuit(circuit, noise_level=0.001):
    """Return bitstrings outcomes from circuit executions performed by
    a density matrix simulation with depolarizing noise."""
    circuit = circuit.with_noise(cirq.depolarize(p=noise_level))
    result = cirq.DensityMatrixSimulator().run(circuit, repetitions=100)
    bitstrings = np.column_stack(list(result.measurements.values()))
    return MeasurementResult(bitstrings)

In [ ]:
def execute_qiskit_circuit(circuit, shots=100):
    """Return bitstrings outcomes from circuit executions performed by
    the FakeJakartaV2 simulator with noise."""
    noisy_backend = FakeJakartaV2()
    noisy_result = noisy_backend.run(circuit, shots=shots).result()
    noisy_counts = noisy_result.get_counts(circuit)
    noisy_counts = { k.replace(" ",""):v for k, v in noisy_counts.items()}
    measurements = MeasurementResult.from_counts(noisy_counts)
    return measurements

## Run the Calibrator
Now that the executor is defined, we can instantiate the Calibrator by passing in the executor along with a `fronted`.

In [11]:
mitiq.SUPPORTED_PROGRAM_TYPES.keys()

['braket', 'cirq', 'pennylane', 'pyquil', 'qibo', 'qiskit']

In [18]:
cal = Calibrator(execute_with_depolarizing, frontend="cirq")
cal.run()

In [ ]:
cal.best_strategy()

In [19]:
cal = Calibrator(execute_with_depolarizing, frontend="cirq")
cal.run(log="flat")

┌──────────────────────────┬────────────────────────────────────┬────────────────────────────┐
│ benchmark                │ strategy                           │ performance                │
├──────────────────────────┼────────────────────────────────────┼────────────────────────────┤
│ Type: ghz                │ Technique: ZNE                     │ ✔                          │
│ Num qubits: 2            │ Factory: Linear                    │ Noisy error: 0.05          │
│ Circuit depth: 2         │ Scale factors: 1.0, 3.0, 5.0       │ Mitigated error: 0.0158    │
│ Two qubit gate count: 1  │ Scale method: fold_global          │ Improvement factor: 3.1579 │
├──────────────────────────┼────────────────────────────────────┼────────────────────────────┤
│ Type: ghz                │ Technique: ZNE                     │ ✔                          │
│ Num qubits: 2            │ Factory: Linear                    │ Noisy error: 0.05          │
│ Circuit depth: 2         │ Scale factors: 1.0, 3

In [17]:
cal.run(log="cartesian")

┌────────────────────────────────────┬────────────────────────────┬────────────────────────────┬────────────────────────────┬────────────────────────────┐
│ strategy\benchmark                 │ Type: ghz                  │ Type: w                    │ Type: rb                   │ Type: mirror               │
│                                    │ Num qubits: 2              │ Num qubits: 2              │ Num qubits: 2              │ Num qubits: 2              │
│                                    │ Circuit depth: 2           │ Circuit depth: 2           │ Circuit depth: 45          │ Circuit depth: 33          │
│                                    │ Two qubit gate count: 1    │ Two qubit gate count: 2    │ Two qubit gate count: 10   │ Two qubit gate count: 14   │
├────────────────────────────────────┼────────────────────────────┼────────────────────────────┼────────────────────────────┼────────────────────────────┤
│ Technique: ZNE                     │ ✘                          │ ✘ 

In [21]:
cal.best_strategy()

{'technique': 'ZNE', 'factory': 'LinearFactory', 'scale_factors': [1.0, 3.0, 5.0], 'scale_method': 'fold_gates_at_random'}

In [24]:
from mitiq.calibration import PEC_SETTINGS

cal = Calibrator(execute_with_depolarizing, frontend="cirq", settings=PEC_SETTINGS)

In [29]:
print(PEC_SETTINGS)

'<mitiq.calibration.settings.Settings object at 0x17b0f6090>'

In [26]:
cal.run(log="flat")

┌──────────────────────────┬───────────────────────────────────────────────────┬────────────────────────────┐
│ benchmark                │ strategy                                          │ performance                │
├──────────────────────────┼───────────────────────────────────────────────────┼────────────────────────────┤
│ Type: ghz                │ Technique: PEC                                    │ ✘                          │
│ Num qubits: 2            │ Representation function: local_depolarizing_noise │ Noisy error: 0.0           │
│ Circuit depth: 2         │ Noise level: 0.01                                 │ Mitigated error: 0.0008    │
│ Two qubit gate count: 1  │ Noise bias: 0                                     │ Improvement factor: 0.0    │
│                          │ Is qubit dependent: False                         │                            │
│                          │ Num samples: 200                                  │                            │
├─────────

In [30]:
cal.best_strategy()

{'technique': 'PEC', 'representation_function': 'represent_operation_with_local_depolarizing_noise', 'noise_level': 0.001, 'noise_bias': 0, 'is_qubit_dependent': False, 'num_samples': 200}